In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.
         appName("working with tabular data").
         getOrCreate())

In [3]:
import os
import pyspark.sql.functions as F

In [4]:
DIRECTORY = r"C:\Users\oluwa\Documents\broadcast_logs"

In [5]:
#%%bash

#!ls DIRECTORY
files = os.listdir(DIRECTORY)

for f in files:
    print(f)
print("-"*100)
for f in files:
    print(os.path.join(DIRECTORY, f))

BroadcastLogs_2018_Q3_M8.CSV
BroadcastLogs_2018_Q3_M8_sample.CSV
data dictionary.doc
ReferenceTables
----------------------------------------------------------------------------------------------------
C:\Users\oluwa\Documents\broadcast_logs\BroadcastLogs_2018_Q3_M8.CSV
C:\Users\oluwa\Documents\broadcast_logs\BroadcastLogs_2018_Q3_M8_sample.CSV
C:\Users\oluwa\Documents\broadcast_logs\data dictionary.doc
C:\Users\oluwa\Documents\broadcast_logs\ReferenceTables


In [6]:
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8.csv"),
    sep = "|",
    header = True,
    inferSchema = True,
    timestampFormat = "yyyy-MM-dd",
)

In [ ]:
logs.printSchema()

In [ ]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)